# Initialization

In [1]:
from keras.models import Sequential, Model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
# import imgaug as ia
# from tqdm import tqdm
# from imgaug import augmenters as iaa
import numpy as np
import pickle
import os, cv2
# from preprocessing import parse_annotation, BatchGenerator
from utils import WeightReader, decode_netout #, draw_boxes
import time

from glob import glob
import gc_util as gc

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

%matplotlib inline

Using TensorFlow backend.


In [2]:
# LABELS = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
# LABELS = ['Fixed-wing Aircraft', 'Small Aircraft', 'Cargo Plane', 'Helicopter', 'Passenger Vehicle', 'Small Car', 'Bus', 'Pickup Truck', 'Utility Truck', 'Truck', 'Cargo Truck', 'Truck w/Box', 'Truck Tractor', 'Trailer', 'Truck w/Flatbed', 'Truck w/Liquid', 'Crane Truck', 'Railway Vehicle', 'Passenger Car', 'Cargo Car', 'Flat Car', 'Tank car', 'Locomotive', 'Maritime Vessel', 'Motorboat', 'Sailboat', 'Tugboat', 'Barge', 'Fishing Vessel', 'Ferry', 'Yacht', 'Container Ship', 'Oil Tanker', 'Engineering Vehicle', 'Tower crane', 'Container Crane', 'Reach Stacker', 'Straddle Carrier', 'Mobile Crane', 'Dump Truck', 'Haul Truck', 'Scraper/Tractor', 'Front loader/Bulldozer', 'Excavator', 'Cement Mixer', 'Ground Grader', 'Hut/Tent', 'Shed', 'Building', 'Aircraft Hangar', 'Damaged Building', 'Facility', 'Construction Site', 'Vehicle Lot', 'Helipad', 'Storage Tank', 'Shipping container lot', 'Shipping Container', 'Pylon', 'Tower']
#LABELS = ['Small Car','Truck','Bus']
LABELS=['Building']
IMAGE_H, IMAGE_W = 416, 416
GRID_H,  GRID_W  = 13, 13   # Vince: changed from 13 to 8
BOX              = 5
CLASS            = len(LABELS)
CLASS_WEIGHTS    = np.ones(CLASS, dtype='float32')
OBJ_THRESHOLD    = 0.5#0.5
NMS_THRESHOLD    = 0.2#0.45
#ANCHORS          = [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]
ANCHORS          = [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]
NO_OBJECT_SCALE  = 1.0
OBJECT_SCALE     = 5.0
COORD_SCALE      = 1.0
CLASS_SCALE      = 1.0

BATCH_SIZE       = 16
WARM_UP_BATCHES  = 0
TRUE_BOX_BUFFER  = 50

# Construct the network

In [3]:
# the function to implement the orgnization layer (thanks to github.com/allanzelener/YAD2K)
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [4]:
input_image = Input(shape=(IMAGE_H, IMAGE_W, 3))
true_boxes  = Input(shape=(1, 1, 1, TRUE_BOX_BUFFER , 4))

# Layer 1
x = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input_image)
x = BatchNormalization(name='norm_1')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Layer 2
x = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(x)
x = BatchNormalization(name='norm_2')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Layer 3
x = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(x)
x = BatchNormalization(name='norm_3')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 4
x = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)(x)
x = BatchNormalization(name='norm_4')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 5
x = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(x)
x = BatchNormalization(name='norm_5')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Layer 6
x = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(x)
x = BatchNormalization(name='norm_6')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 7
x = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_7', use_bias=False)(x)
x = BatchNormalization(name='norm_7')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 8
x = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(x)
x = BatchNormalization(name='norm_8')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Layer 9
x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(x)
x = BatchNormalization(name='norm_9')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 10
x = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_10', use_bias=False)(x)
x = BatchNormalization(name='norm_10')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 11
x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(x)
x = BatchNormalization(name='norm_11')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 12
x = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_12', use_bias=False)(x)
x = BatchNormalization(name='norm_12')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 13
x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)(x)
x = BatchNormalization(name='norm_13')(x)
x = LeakyReLU(alpha=0.1)(x)

skip_connection = x

x = MaxPooling2D(pool_size=(2, 2))(x)

# Layer 14
x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(x)
x = BatchNormalization(name='norm_14')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 15
x = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_15', use_bias=False)(x)
x = BatchNormalization(name='norm_15')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 16
x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(x)
x = BatchNormalization(name='norm_16')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 17
x = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_17', use_bias=False)(x)
x = BatchNormalization(name='norm_17')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 18
x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(x)
x = BatchNormalization(name='norm_18')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 19
x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(x)
x = BatchNormalization(name='norm_19')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 20
x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(x)
x = BatchNormalization(name='norm_20')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 21
skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='norm_21')(skip_connection)
skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
skip_connection = Lambda(space_to_depth_x2)(skip_connection)

x = concatenate([skip_connection, x])

# Layer 22
x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(x)
x = BatchNormalization(name='norm_22')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 23
x = Conv2D(BOX * (4 + 1 + CLASS), (1,1), strides=(1,1), padding='same', name='conv_23')(x)
output = Reshape((GRID_H, GRID_W, BOX, 4 + 1 + CLASS))(x)

# small hack to allow true_boxes to be registered when Keras build the model 
# for more information: https://github.com/fchollet/keras/issues/2790
output = Lambda(lambda args: args[0])([output, true_boxes])

model = Model([input_image, true_boxes], output)

In [5]:
# model.summary()

**Parse the annotations to construct train generator and validation generator**

In [6]:
generator_config = {
    'IMAGE_H'         : IMAGE_H, 
    'IMAGE_W'         : IMAGE_W,
    'GRID_H'          : GRID_H,  
    'GRID_W'          : GRID_W,
    'BOX'             : BOX,
    'LABELS'          : LABELS,
    'CLASS'           : len(LABELS),
    'ANCHORS'         : ANCHORS,
    'BATCH_SIZE'      : BATCH_SIZE,
    'TRUE_BOX_BUFFER' : 50,
}

# Perform detection on image

In [7]:
model.load_weights("weights_building.h5")

In [8]:
val_ids = [10130002, 10210001, 10210002, 10210012, 10210013, 10210028, 10210031, 10210034, 10330003]

In [9]:
from hyper import IMG_ID as img_id
from hyper import NUM_SQ
from hyper import OBJ_THRESH as obj_thresh
from hyper import WH
COORDS = [(i, j) for i in range(0,1000,WH) for j in range(0,1000,WH)]

In [10]:
def get_files(img_id):
    sqs = [i for i in range(1, NUM_SQ+1)]
    gc.open_images(img_id, sqs)

    img_files = 'LSMS_dg/dg_lsms_uganda_1000x1000_' + str(img_id) + '_*.jpeg'
    val_files = glob(img_files)

    def fn(name):
        return int(name[42:-5])  # parsing out the sq variable

    val_files = sorted(val_files, key=fn)
    val_files = val_files[:NUM_SQ]
    print("Acquired files for %d" %img_id)
    
    return val_files

In [29]:
def object_detection(img_id, val_files, verbose=False):
    # assumes that img_id matches the val_files
    
    base = "boxes/%d" %img_id
    if not os.path.exists(base):
        os.makedirs(base)
    
    print("Performing object detection at %d on %s tiles, each %s chips of w,h=%s" % (img_id, NUM_SQ, int((1000/WH)**2), WH))
    if verbose: print()
    
    start = time.time()
    for k, img_path in enumerate(val_files):
        
        # check if file has already been scanned
        tst_name = base + '/' + img_path[8:-5] + '_' + str(int((1000/WH)**2) - 1) + '.pickle'
        if os.path.isfile(tst_name):
            continue
        
        # measuring progress if so desired
        if verbose:
            if (k+1) % 100 == 0:
                pdone = float(k) / len(val_files)
                curr_time = time.time()
                rem = int((1-pdone)/pdone * (curr_time - start))
                print("%.2f percent done at sq %s. ETC: %s:%02d" % (100 * pdone, k+1, int(rem / 60), rem % 60))

        image = cv2.imread(img_path)

        # if unable to load image, redownload and try again
        if image is None:
            if os.path.exists(img_path):
                os.remove(img_path)
            gs.open_images(img_id, k+1)
            image = cv2.imread(img_path)
            if image is None:
                print("Error processing %s, continuing with rest of objects" %img_path)
                continue
        
        for idx, coord_pair in enumerate(COORDS):
            image_quarter = image[coord_pair[0]:coord_pair[0]+WH,
                                  coord_pair[1]:coord_pair[1]+WH]

            dummy_array = np.zeros((1,1,1,1,TRUE_BOX_BUFFER,4))
            input_image = cv2.resize(image_quarter, (416, 416))
            input_image = input_image / 255.
            input_image = input_image[:,:,::-1]  # reversing RGB channels
            input_image = np.expand_dims(input_image, 0)
            netout = model.predict([input_image, dummy_array])

            boxes = decode_netout(netout[0],
                              obj_threshold=obj_thresh,
                              nms_threshold=NMS_THRESHOLD,
                              anchors=ANCHORS,
                              nb_class=CLASS)

            for box in boxes:
                image_h, image_w, _ = image_quarter.shape

                box.xmin *= image_w
                box.ymin *= image_h
                box.xmax *= image_w
                box.ymax *= image_h

            # now separating into individual folders for cleanliness
            out_name = base + '/' + img_path[8:-5] + '_' + str(idx) + '.pickle'
            pickle_out = open(out_name, "wb")
            pickle.dump(boxes, pickle_out)
            pickle_out.close()
            
    if verbose: print()
    print("Object detection completed!")
    print()

In [30]:
for img_id in val_ids:
    val_files = get_files(img_id)
    object_detection(img_id, val_files, verbose=False)

Acquired files for 10130002
Performing object detection at 10130002 on 1156 tiles, each 4 chips of w,h=500

Error processing LSMS_dg/dg_lsms_uganda_1000x1000_10130002_662.jpeg, continuing with rest of objects
60.47 percent done at sq 700. ETC: 0:48
69.12 percent done at sq 800. ETC: 2:13
77.77 percent done at sq 900. ETC: 2:28
86.42 percent done at sq 1000. ETC: 1:56

Object detection completed!

Acquired files for 10210001
Performing object detection at 10210001 on 1156 tiles, each 4 chips of w,h=500

8.56 percent done at sq 100. ETC: 39:56
17.21 percent done at sq 200. ETC: 36:16
25.87 percent done at sq 300. ETC: 32:30
34.52 percent done at sq 400. ETC: 28:43
43.17 percent done at sq 500. ETC: 24:54
51.82 percent done at sq 600. ETC: 21:04
60.47 percent done at sq 700. ETC: 17:16
69.12 percent done at sq 800. ETC: 13:28
77.77 percent done at sq 900. ETC: 9:41
86.42 percent done at sq 1000. ETC: 5:54
95.07 percent done at sq 1100. ETC: 2:08

Object detection completed!

Acquired file

OSError: [Errno 28] No space left on device

In [81]:
from hyper import HOUSE_SIZE_THRESH
from hyper import HOUSE_LEN_THRESH

def draw_boxes_coords(img_id, sq, coords=COORDS, filter_size=True, filter_green=True):
    img_path = "LSMS_dg/dg_lsms_uganda_1000x1000_" + str(img_id) + "_" + str(sq) + ".jpeg"
    
    imgs = []
    
    if sq % 50 == 1:
        print("Currently at sq %d" %sq)
    
    for idx, coord_pair in enumerate(coords):
        pickle_path = "boxes/" + str(img_id) + "/dg_lsms_uganda_1000x1000_" + str(img_id) + "_" + str(sq) + "_" + str(idx) + ".pickle"

        pickle_in = open(pickle_path, "rb")
        boxes = pickle.load(pickle_in)

        img = cv2.imread(img_path)
        img = img[coord_pair[0]:coord_pair[0]+WH,
                  coord_pair[1]:coord_pair[1]+WH]

        for tbox in boxes:
            if filter_size:
                size = (tbox.xmax - tbox.xmin) * (tbox.ymax - tbox.ymin)
                def bad_house_size(tbox):
                    return size > HOUSE_SIZE_THRESH or (tbox.xmax - tbox.xmin) > HOUSE_LEN_THRESH or (tbox.ymax - tbox.ymin) > HOUSE_LEN_THRESH
                if bad_house_size(tbox): continue
            
            p1 = (int(tbox.xmin), int(tbox.ymin))
            p2 = (int(tbox.xmax), int(tbox.ymax))
            
            if filter_green:
                reg = np.array(img[p1[1]:p2[1], p1[0]:p2[0]])
                # plt.imshow(reg)
                # plt.show()
                
                def bad_green_maj(reg, thresh=0.70):
                    num_pix = (p2[1] - p1[1]) * (p2[0] - p1[0])
                    green = reg[:,:,1]
                    mxd = np.amax(reg, axis=2)
                    num_nongreen = np.count_nonzero(green - mxd)
                    pc = 1 - float(num_nongreen) / num_pix
                    # print(pc)
                    return pc > thresh
                
                if bad_green_maj(reg): continue
                
            img = cv2.rectangle(img, p1, p2, (255,0,0), thickness=1)
    
        imgs.append(img)
    
    return imgs

In [82]:
def save_boxes_coords(img_id, sq, coords=COORDS, filter_size=True, filter_green=True):
    imgs = draw_boxes_coords(img_id, sq, coords=COORDS, filter_size=filter_size, filter_green=filter_green)
    
    if WH == 250:
        rws = []
        for k in range(0,16,4):
            rws.append(np.concatenate([imgs[k], imgs[k+1], imgs[k+2], imgs[k+3]], axis=1))
        img = np.concatenate([rws[0], rws[1], rws[2], rws[3]], axis=0)
        path = "box_imgs/dg_lsms_uganda_1000x1000_%s_%s_boxed.jpeg" % (img_id, sq)
        cv2.imwrite(path, img)
    elif WH == 500:
        for idx in range(len(coords)):
            path = "box_imgs/dg_lsms_uganda_1000x1000_" + str(img_id) + "_" + str(sq) + "_" + str(idx) + ".jpeg"
            cv2.imwrite(path, imgs[idx])

In [83]:
for sq in range(1, NUM_SQ+1):
    save_boxes_coords(img_id, sq, filter_size=True, filter_green=True)

Currently at sq 1
Currently at sq 51
Currently at sq 101
Currently at sq 151
Currently at sq 201
Currently at sq 251
Currently at sq 301
Currently at sq 351
Currently at sq 401
Currently at sq 451
Currently at sq 501
Currently at sq 551
Currently at sq 601
Currently at sq 651
Currently at sq 701
Currently at sq 751
Currently at sq 801
Currently at sq 851
Currently at sq 901
Currently at sq 951
Currently at sq 1001
Currently at sq 1051
Currently at sq 1101
Currently at sq 1151


# Delete folders for restart

In [31]:
def empty_folder(folder):
    import shutil
    if os.path.exists(folder):
        shutil.rmtree(folder)
        os.makedirs(folder)

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [87]:
empty_folder("LSMS_dg")

In [85]:
empty_folder("box_imgs")

In [86]:
empty_folder("boxes")